# Exercise: MA(1) Model

In [ ]:
using DSGE
using DataFrames, DataStructures, Distributions, Plots
include("util.jl")

# This file defines the MA1 type
include("ma1.jl")

In [ ]:
# See docstring for MA1
?MA1

In [ ]:
# Initialize MA(1) model
m = MA1()

m <= Setting(:saveroot, dirname(@__FILE__))
m <= Setting(:date_presample_start, DSGE.quartertodate("2001-Q1"))
m <= Setting(:date_mainsample_start, DSGE.quartertodate("2001-Q1"))
m <= Setting(:date_forecast_start, DSGE.quartertodate("2026-Q1"))
m <= Setting(:n_mh_simulations, 500)
m <= Setting(:n_mh_blocks, 10)
m <= Setting(:n_mh_burn, 2)

In [ ]:
m.parameters

In [ ]:
m.endogenous_states

In [ ]:
m.exogenous_shocks

In [ ]:
m.observables

## Exercise 1

Given the following definition of `eqcond` (which returns the equilibrium condition matrices for an `MA1` model), fill in the `ZZ` and `QQ` matrices in `measurement`.

In [ ]:
function DSGE.eqcond(m::MA1)
    endo = m.endogenous_states
    exo  = m.exogenous_shocks
    eq   = m.equilibrium_conditions

    Γ0 = zeros(n_states(m), n_states(m))
    Γ1 = zeros(n_states(m), n_states(m))
    C  = zeros(n_states(m))
    Ψ  = zeros(n_states(m), n_shocks_exogenous(m))
    Π  = zeros(n_states(m), n_shocks_expectational(m))

    # Γ0*s_t = Γ1*s_{t-1} + Ψ*ϵ_t + Π*η_t + C
    # s_t = [u_t, u_{t-1}]'
    # ϵ_t = [u_t]'

    # Row 1: u_t = 0*u_{t-1} + 0*u_{t-2} + u_t
    Γ0[eq[:eq_u_t], endo[:u_t]] = 1
     Ψ[eq[:eq_u_t],  exo[:u_t]] = 1

    # Row 2: u_{t-1} = u_{t-1} + 0 u_{t-2} + 0 u_t
    Γ0[eq[:eq_u_t1], endo[:u_t1]] = 1
    Γ1[eq[:eq_u_t1], endo[:u_t]]  = 1

    return Γ0, Γ1, C, Ψ, Π
end

In [ ]:
function DSGE.measurement{T<:AbstractFloat}(m::MA1{T}, TTT::Matrix{T}, RRR::Matrix{T}, CCC::Vector{T};
                                            shocks::Bool = false)
    endo = m.endogenous_states # OrderedDict{Symbol, Int} mapping state names (e.g. `:u_t`) to indices
    exo  = m.exogenous_shocks  # ... mapping shock names to indices
    obs  = m.observables       # ... mapping observable names to indices

    ZZ = zeros(n_observables(m), n_states(m))
    DD = zeros(n_observables(m))
    MM = zeros(n_observables(m), n_shocks_exogenous(m))
    EE = zeros(n_observables(m), n_observables(m))
    QQ = zeros(n_shocks_exogenous(m), n_shocks_exogenous(m))

    # y_t = Z*s_t + D
    # y_t = [x_t]'
    # s_t = [u_t, u_{t-1}]'

    # TODO: fill in entries of ZZ matrix
    # x_t = μ + u_t + β*u_{t-1}
    DD[obs[:x_t]] = m[:μ]
    ZZ[obs[:x_t], ...] = ...

    # TODO: fill in entries of QQ matrix
    QQ[exo[:u_t], exo[:u_t]] = ...

    QQ[exo[:u_t], exo[:u_t]] = m[:σ]^2

    HH    = EE + MM*QQ*MM'
    VV    = QQ*MM'
    VVall = [[RRR*QQ*RRR' RRR*VV];
             [VV'*RRR'    HH]]

    return Measurement(ZZ, DD, QQ, EE, MM, VVall)
end

## Exercise 2

Generate 4000 draws from the prior distribution of each parameter.

In [ ]:
# Hint: use the `rand` function, which can be called on any `Distribution`.
dist_μ = get(m[:μ].prior)
dist_β = get(m[:β].prior)
dist_σ = get(m[:σ].prior)

isa(dist_μ, Distribution)

In [ ]:
prior_draws = zeros(4000, 3)
for i = 1:4000
    # TODO: fill in `prior_draws`
end

In [ ]:
# Plot histograms of prior draws
prior_plot = plot(prior_draws, 
                  layout = @layout([a b c]),
                  label = [m[:μ].tex_label m[:β].tex_label m[:σ].tex_label],
                  t     = [:histogram      :histogram      :histogram],
                  color = [:red            :blue           :green],
                  title = ["" "Prior" ""])

## Exercise 3

Given data generated below, sample from the posterior distribution of the parameters. (No new code needs to be written in this section.)

In [ ]:
# True values of parameters
μ = 0.75
β = 0.9
σ = 0.25

dist_u = Normal(0, σ)

# Initialize states
u_t  = 0.0
u_t1 = 0.0

# Initialize DataFrame with 100 periods (2001-Q1 to 2025-Q4)
df = DataFrame(date = DSGE.quarter_range(DSGE.quartertodate("2001-Q1"), DSGE.quartertodate("2025-Q4")))
df[:x_t] = NaN

for t = 1:100
    # Set last period's u_t value to this period's u_{t-1}
    u_t1 = u_t

    # Draw new value of u_t
    u_t = rand(dist_u)

    # Apply measurement equation to get x_t
    x_t = μ + u_t + β*u_t1

    # Record in DataFrame
    df[t, :x_t] = x_t
end

In [ ]:
# Find posterior mode and sample using Metropolis-Hastings
# This function call will take a minute to run the first time
estimate(m, df)

In [ ]:
# Load posterior draws
post_draws = load_draws(m, :full)

# Plot posterior draws
post_plot = plot(post_draws,
                 layout = @layout([a b c]),
                 label = [m[:μ].tex_label m[:β].tex_label m[:σ].tex_label],
                 t     = [:histogram      :histogram      :histogram],
                 color = [:red            :blue           :green],
                 title = ["" "Posterior" ""])

# Plot actual values
plot_actual!(post_plot.subplots[1], μ)
plot_actual!(post_plot.subplots[2], β)
plot_actual!(post_plot.subplots[3], σ)

In [ ]:
# Plot actuals on top of prior
plot_actual!(prior_plot.subplots[1], μ)
plot_actual!(prior_plot.subplots[2], β)
plot_actual!(prior_plot.subplots[3], σ)

# Plot both prior and posterior together
all_plots = plot(prior_plot, post_plot, layout = @layout([a; b]))